In [5]:
import os
import numpy as np
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet import preprocess_input
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

# Initialize the pre-trained MobileNet model without the top classification layer
model = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3), pooling='avg')

# Function to extract features from an image
def extract_features(img_path, model):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)  # Preprocess for MobileNet
    features = model.predict(img_array)
    return features.flatten()

# Directory paths
omelette_train_dir = '../data/food11/train/omelette'
omelette_test_dir = '../data/food11/test/omelette'

# Collect all image paths in the omelette folders
image_paths = [os.path.join(omelette_train_dir, fname) for fname in os.listdir(omelette_train_dir)]
image_paths += [os.path.join(omelette_test_dir, fname) for fname in os.listdir(omelette_test_dir)]

# Extract features for all images
features = []
for img_path in image_paths:
    try:
        features.append(extract_features(img_path, model))
    except Exception as e:
        print(f"Error processing image {img_path}: {e}")

features = np.array(features)
print(f"Extracted features for {len(features)} images")

Extracted features for 995 images


In [2]:
from sklearn.cluster import KMeans

# Number of clusters (choose based on expected subcategories, e.g., 3 types of pizza)
num_clusters = 6

# Perform K-Means clustering
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans.fit(features)

# Get cluster labels for each image
cluster_labels = kmeans.labels_

# Print the cluster label for each image
for img_path, label in zip(image_paths, cluster_labels):
    print(f"{img_path} assigned to cluster {label}")

c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


../data/food11/train/omelette\1001719.jpg assigned to cluster 0
../data/food11/train/omelette\1007843.jpg assigned to cluster 1
../data/food11/train/omelette\1015300.jpg assigned to cluster 5
../data/food11/train/omelette\1019294.jpg assigned to cluster 1
../data/food11/train/omelette\1029915.jpg assigned to cluster 2
../data/food11/train/omelette\1038109.jpg assigned to cluster 5
../data/food11/train/omelette\1048878.jpg assigned to cluster 3
../data/food11/train/omelette\1053091.jpg assigned to cluster 5
../data/food11/train/omelette\1058735.jpg assigned to cluster 4
../data/food11/train/omelette\1058816.jpg assigned to cluster 2
../data/food11/train/omelette\1063810.jpg assigned to cluster 3
../data/food11/train/omelette\1063811.jpg assigned to cluster 5
../data/food11/train/omelette\1079426.jpg assigned to cluster 4
../data/food11/train/omelette\1090229.jpg assigned to cluster 1
../data/food11/train/omelette\1091433.jpg assigned to cluster 4
../data/food11/train/omelette\1097876.jp

In [6]:
import shutil

# Create directories for each cluster (subcategories)
for i in range(num_clusters):
    train_subdir = os.path.join(omelette_train_dir, f"cluster_{i}")
    test_subdir = os.path.join(omelette_test_dir, f"cluster_{i}")
    os.makedirs(train_subdir, exist_ok=True)
    os.makedirs(test_subdir, exist_ok=True)

# Move images to subdirectories based on cluster labels
for img_path, label in zip(image_paths, cluster_labels):
    if 'train' in img_path:
        shutil.move(img_path, os.path.join(omelette_train_dir, f"cluster_{label}", os.path.basename(img_path)))
    elif 'test' in img_path:
        shutil.move(img_path, os.path.join(omelette_test_dir, f"cluster_{label}", os.path.basename(img_path)))

print("Images moved to their respective cluster directories.")

Images moved to their respective cluster directories.


In [12]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model

# Fine-grained apple_pie classification
omelette_train_gen = ImageDataGenerator(rescale=1./255)
omelette_test_gen = ImageDataGenerator(rescale=1./255)

omelette_train_generator = omelette_train_gen.flow_from_directory(
    omelette_train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

omelette_test_generator = omelette_test_gen.flow_from_directory(
    omelette_test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

# Use the same base model (MobileNet)
base_model_omelette = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model layers
for layer in base_model_omelette.layers:
    layer.trainable = False

# Add custom layers for apple_pie classification
x_omelette = base_model_omelette.output
x_omelette = GlobalAveragePooling2D()(x_omelette)
x_omelette = Dense(512, activation='relu')(x_omelette)
omelette_predictions = Dense(num_clusters, activation='softmax')(x_omelette)

# Create and compile the model
omelette_model = Model(inputs=base_model_omelette.input, outputs=omelette_predictions)
omelette_model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the fine-grained apple_pie classification model
omelette_model.fit(
    omelette_train_generator,
    steps_per_epoch=omelette_train_generator.samples // 32,
    validation_data=omelette_test_generator,
    validation_steps=omelette_test_generator.samples // 32,
    epochs=30
)

# Save the model
omelette_model.save('fine_grained_omelette_classifier.h5')

Found 900 images belonging to 6 classes.
Found 95 images belonging to 6 classes.
Epoch 1/30
28/28 [==============================] - 8s 258ms/step - loss: 1.4573 - accuracy: 0.4482 - val_loss: 1.7165 - val_accuracy: 0.3281
Epoch 2/30
28/28 [==============================] - 7s 252ms/step - loss: 0.8759 - accuracy: 0.7362 - val_loss: 1.6577 - val_accuracy: 0.4375
Epoch 3/30
28/28 [==============================] - 7s 248ms/step - loss: 0.6154 - accuracy: 0.8433 - val_loss: 1.7017 - val_accuracy: 0.5000
Epoch 4/30
28/28 [==============================] - 7s 249ms/step - loss: 0.4568 - accuracy: 0.8917 - val_loss: 1.8712 - val_accuracy: 0.4844
Epoch 5/30
28/28 [==============================] - 7s 249ms/step - loss: 0.3662 - accuracy: 0.9332 - val_loss: 1.9997 - val_accuracy: 0.5000
Epoch 6/30
28/28 [==============================] - 7s 248ms/step - loss: 0.2961 - accuracy: 0.9562 - val_loss: 2.0321 - val_accuracy: 0.5156
Epoch 7/30
28/28 [==============================] - 7s 248ms/step -